In [5]:
def u3D(r,z,E=100e6,nu=0.3,F=100e3):
    Rad = np.sqrt(r**2 + z**2)
    G = E/(2*(1+nu))
    fak = F/(4*np.pi*G*Rad)
    return fak * (2*(1-nu) + (z/Rad)**2)

In [6]:
def u1D(r,z,E=100e6,nu=0.3,F=100e3):
    Rad = np.sqrt(r**2 + z**2)
    Es = E*(1-nu)/((1+nu)*(1-2*nu))
    fak = F/(2*np.pi*Es*Rad)
    return fak * (2 + (z/Rad)**2)

In [ ]:
from ipywidgets import widgets
from ipywidgets import interact
def interactive_point_settlement():
    @interact(#c=widgets.BoundedFloatText(value=0.3,min=0, description='$c$ / MPa'),
               E=widgets.BoundedFloatText(value=100, min=1e-3, max=1e3, description='$E$ / MPa'),
               F=widgets.BoundedFloatText(value=100, min=1, max=1e4, description='$F_z$ / kN'),
               nu=widgets.BoundedFloatText(value=0.3, min=0, max=0.49, description='$\\nu$')
    )
    def plot_MC_states(E,F,nu):
        E *= 1e6
        F *= 1e3
        z = np.array([0.1,0.5,1,2,3,5]) # m
        r = np.linspace(0,5,100)
        sns.set_palette("Paired",n_colors=12)
        fig, ax = plt.subplots(figsize=(18,8),ncols=2)
        for i in z:
            ax[0].plot(r,-u3D(r,i,E,nu,F)*1e3-i,label="$z = %.2f$ m" %i)
            ax[0].plot(r,-u1D(r,i,E,nu,F)*1e3-i,label="$z = %.2f$ m" %i,ls='--')
            ax[0].axhline(-i,ls=':',color='black')

        ax[0].set_xlabel('$r$ / m')
        ax[0].set_ylabel('$u_z$ / mm + $z$ / m')
        ax[0].set_yticks([0,-1,-2,-3,-4,-5])
        ax[0].set_yticklabels([0,1,2,3,4,5])
        ax[0].set_xlim(0,5)
        ax[0].spines['bottom'].set_position(('data', 0))
        ax[0].xaxis.set_label_position('top') 
        ax[0].xaxis.tick_top()
        #
        z = np.linspace(0.1,5,100)
        ax[1].plot(z,u3D(0,z,E,nu,F)*1e3,label='3D solution')
        ax[1].plot(z,u1D(0,z,E,nu,F)*1e3,label='$\\epsilon_{xx}=\\epsilon_{yy} = 0$',ls='--')
        ax[1].set_xlabel('$z$ / m')
        ax[1].set_ylabel('$u_z$ / mm')
        ax[1].set_xlim(left=0)
        ax[1].set_ylim(bottom=0)
        ax[1].legend()
        fig.tight_layout();

In [ ]:
def u3D_Rechteck(t,a,b,E=100e6,nu=0.3,s0=100.e3):
    Rad = np.sqrt(a**2+b**2+t**2)
    fak1 = s0*(1-nu**2)/(np.pi * E)
    fak2 = s0*(1-nu-2*nu**2)/(2*np.pi * E)
    t1 = a*np.log( (b + np.sqrt(a**2+b**2)) * np.sqrt(a**2+t**2) / (a*(b+Rad)) )
    t2 = b*np.log( (a + np.sqrt(a**2+b**2)) * np.sqrt(b**2+t**2) / (b*(a+Rad)) )
    t3 = t * np.arctan(a*b/(t*Rad))
    return fak1 * (t1 + t2) + fak2 * t3

In [ ]:
def u3D_Kreis(t,R,E,nu,s0):
    wrzl = np.sqrt(t**2 + R**2)
    t1 = (1-nu**2)*(R+t-wrzl)
    t2 = t*(1+nu)/2 * (1-t/wrzl)
    return 2*s0/E * (t1-t2)

In [ ]:
def interactive_rect_settlement():
    @interact(#c=widgets.BoundedFloatText(value=0.3,min=0, description='$c$ / MPa'),
               E=widgets.BoundedFloatText(value=100, min=1e-3, max=1e3, description='$E$ / MPa'),
               s0=widgets.BoundedFloatText(value=100, min=1, max=1e4, description='$q_0$ / kPa'),
               a=widgets.BoundedFloatText(value=3, min=0.1, max=10, description='$a$ / m'),
               b=widgets.BoundedFloatText(value=2, min=0.1, max=10, description='$b$ / m')
    )
    def plot_MC_states(E,s0,a,b):
        E *= 1e6
        s0 *=1e3
        t = np.linspace(0.1,100,1000) # m
        sns.set_palette("rocket")
        fig, ax = plt.subplots(figsize=(12,8))
        for nu in [0.0,0.1,0.2,0.3,0.4,0.5]:
            ax.plot(t,u3D_Rechteck(t,a,b,E,nu,s0)*1e3,label='$\\nu = %.1f$' %nu)
        ax.set_xlabel('$t$ / m')
        ax.set_ylabel('$s$ / mm')
        ax.set_xlim(left=0)
        ax.set_ylim(bottom=0)
        ax.legend()
        fig.tight_layout();

In [1]:
def interactive_rect_circ_settlement():
    @interact(E=widgets.BoundedFloatText(value=100, min=1e-3, max=1e3, description='$E$ / MPa'),
              s0=widgets.BoundedFloatText(value=100, min=1, max=1e4, description='$q_0$ / kPa'),
              a=widgets.BoundedFloatText(value=3, min=0.1, max=10, description='$a$ / m'),
              b=widgets.BoundedFloatText(value=2, min=0.1, max=10, description='$b$ / m')
    )
    def plot_MC_states(E,s0,a,b):
        E *= 1e6
        s0 *=1e3
        R = np.sqrt(a*b/np.pi)
        t = np.linspace(0.1,100,1000) # m
        colors = sns.color_palette("rocket").as_hex()
        fig, ax = plt.subplots(figsize=(12,8))
        for i,nu in enumerate([0.15,0.3,0.45]):
            uMR = 4*u3D_Rechteck(t,a/2,b/2,E,nu,s0)*1e3
            uER = u3D_Rechteck(t,a,b,E,nu,s0)*1e3
            uMK = u3D_Kreis(t,R,E,nu,s0)*1e3
            ax.plot(t,uMR,label='$\\nu = %.1f$' %nu,color=colors[i])
            ax.plot(t,uER,color=colors[i],ls=':')
            ax.plot(t,uMK,color=colors[i],ls='--')
        ax.set_xlabel('$t$ / m')
        ax.set_ylabel('$s$ / mm')
        ax.plot(0,0,color='black',ls='-',label='Rechteck, Mitte')
        ax.plot(0,0,color='black',ls=':',label='Rechteck, Eckpunkt')
        ax.plot(0,0,color='black',ls='--',label='Kreis mit R = %.3f m, Mitte' %R)
        ax.set_xlim(left=0)
        ax.set_ylim(bottom=0)
        ax.legend(ncol=2)
        fig.tight_layout();

In [ ]:
def i_Rechteck(z,a,b):
    R = np.sqrt(a**2 + b**2 + z**2)
    term1 = b*z/(b**2+z**2) * a*(R**2 + z**2)/((a**2+z**2)*R)
    term2 = np.arctan( b/z * (a*(a**2+b**2) - 2*a*z * (R-z))/((a**2+b**2)*(R-z) - z*(R-z)**2) )
    return (term1+term2)/(2*np.pi)

In [ ]:
def u3D_inf(a,b,E=100e6,nu=0.3,s0=100.e3):
    fak1 = s0*(1-nu**2)/(np.pi * E)
    t1 = a*np.log( (b + np.sqrt(a**2+b**2))/ a )
    t2 = b*np.log( (a + np.sqrt(a**2+b**2))/ b )
    return fak1 * (t1 + t2)

In [2]:
import scipy as scp
def interactive_grenztiefe():
    @interact(a=widgets.BoundedFloatText(value=3, min=0.1, max=10, description='$a$ / m'),
              b=widgets.BoundedFloatText(value=2, min=0.1, max=10, description='$b$ / m'),
              s0=widgets.BoundedFloatText(value=100, min=1, max=1e4, description='$q_0$ / kPa'),
              teval=widgets.BoundedFloatText(value=10, min=0.1, max=100, description='$t_\\mathrm{s}$ / m'),
              n=widgets.BoundedFloatText(value=0.3, min=0, max=0.49, description='$\\nu$'),
              gamma=widgets.BoundedFloatText(value=18, min=16, max=24, description='$\\gamma$ / kPa m$^{-1}$')
    )
    def plot_MC_states(a,b,s0,teval,n,gamma):
        t = np.linspace(1,100,1000) # m
        sns.set_palette("tab10")
        fig, ax = plt.subplots(figsize=(12,8))
        s_ratio = lambda ts: u3D_Rechteck(ts,a,b,s0=s0,nu=n)/u3D_inf(a,b,s0=s0,nu=n)*100
        sig_ratio = lambda ts: i_Rechteck(ts,a,b)*s0 / (gamma * ts) *100
        ax.plot(t,s_ratio(t),label='settlement ratio')
        ax.plot(t,sig_ratio(t),label='stress ratio')
        ax.plot(teval,s_ratio(teval),marker='o',color='red')
        ax.plot(teval,sig_ratio(teval),marker='o',color='red')
        ax.plot([teval,teval],[0,np.maximum(s_ratio(teval),s_ratio(teval))],ls='--',color='black')
        ax.plot([teval,0],[s_ratio(teval),s_ratio(teval)],ls='--',color='black')
        ax.plot([teval,0],[sig_ratio(teval),sig_ratio(teval)],ls='--',color='black')
        ax.text(teval,s_ratio(teval)+5,"%.1f" %s_ratio(teval))
        ax.text(teval,sig_ratio(teval)+5,"%.1f" %sig_ratio(teval))
        limit_f = lambda z: u3D_Rechteck(z,a,b,nu=n)/u3D_inf(a,b,nu=n) - 0.95
        res = scp.optimize.root(fun=limit_f,x0=10)
        ax.set_xlabel('$t$ / m')
        ax.set_ylabel('ratio / %')
        ax.set_xlim(left=0,right=res.x)
        ax.set_ylim(bottom=0)
        ax.legend()
        fig.tight_layout();

In [2]:
def fR(t,a,b,nu=0):
    Rad = np.sqrt(a**2+b**2+t**2)
    fak1 = (1-nu**2)/(b*np.pi)
    fak2 = (1-nu-2*nu**2)/(2*b*np.pi)
    t1 = a*np.log( (b + np.sqrt(a**2+b**2)) * np.sqrt(a**2+t**2) / (a*(b+Rad)) )
    t2 = b*np.log( (a + np.sqrt(a**2+b**2)) * np.sqrt(b**2+t**2) / (b*(a+Rad)) )
    t3 = t * np.arctan(a*b/(t*Rad))
    return fak1 * (t1 + t2) + fak2 * t3

In [2]:
def fK(t,a,b,nu=0):
    ai = [0.87*a,0.87*a,0.87*b,0.13*a]
    bi = [0.87*b,0.13*b,0.13*a,0.13*b]
    f = 0
    for i in range(4):
        f += fR(t,ai[i],bi[i],nu)*bi[i]/b
    return f

In [ ]:
def interactive_fR_rechteck():
    @interact(#c=widgets.BoundedFloatText(value=0.3,min=0, description='$c$ / MPa'),
               z_to_b=widgets.BoundedFloatText(value=1, min=0.1, max=100., description='$z$ / $b$'),
               a_to_b=widgets.BoundedFloatText(value=2, min=0.1, max=100., description='$a$ / $b$')
    )
    def plot_MC_states(z_to_b,a_to_b):
        plt.rcParams['axes.spines.right'] = True
        fig, ax = plt.subplots(figsize=(10,10))
        sns.set_palette("Paired",n_colors=12)
        ax2 = ax.twiny()
        z1 = np.logspace(-1,2,100)
        z2 = np.logspace(-1,0,100)
        for a in [0.1,0.2,0.4,0.8,1,1.5,2,3,5,10,100]:
            ax.plot(fR(z1,a,1),-z1,label='$%.1f$' %a)
            ax2.plot(fR(z2,a,1),-z2)
        
        f = fR(z_to_b,a_to_b,1)
        ax.set_xlim(left=0)
        ax.set_ylim(-100,-0.1)
        ax2.set_xlim(left=0)
        ax.grid(which='both')
        ax.set_yscale('symlog',linthresh=0.01)
        ax.set_yticks([-0.1,-1,-10,-100])
        ax.set_yticklabels([0.1,1,10,100])
        ax.set_yticks(-np.logspace(-1,2,10), minor=True)
        ax.set_xlabel('$f_\\mathrm{R}$')
        ax2.set_xlabel('$f_\\mathrm{R}$')
        ax.set_ylabel('$z$ / $b$')

        if (z_to_b < 1):
            ax2.plot(fR(z2,a_to_b,1),-z2,label='$a$ / $b = %.1f$' %a_to_b,color='black',ls='--')
            ax2.plot([0.,f,f],[-z_to_b,-z_to_b,-0.1],color='black',ls=':')
            ax2.plot(f,-z_to_b,marker='o',color='red')
        else: 
            ax.plot(fR(z1,a_to_b,1),-z1,label='$a$ / $b = %.1f$' %a_to_b,color='black',ls='--')
            ax.plot([0,f,f],[-z_to_b,-z_to_b,-100],color='black',ls=':')
            ax.plot(f,-z_to_b,marker='o',color='red')
        ax.text(0.8,-2,'$s = %.6f \\sigma_0 b/E^*$' %f)
        ax2.spines['bottom'].set_position(('data', -0.1))
        ax2.xaxis.set_label_position('top') 
        ax2.xaxis.tick_top()
        fig.legend(bbox_to_anchor=(1.15, 0.8))
        #fig.tight_layout()
        plt.show();

In [ ]:
def interactive_fR_rechteck_nu():
    @interact(z_to_b=widgets.BoundedFloatText(value=1, min=0.1, max=100., description='$z$ / $b$'),
              a_to_b=widgets.BoundedFloatText(value=2, min=0.1, max=100., description='$a$ / $b$'),
              nu=widgets.BoundedFloatText(value=0.3, min=0, max=0.49, description='$\\nu$')
    )
    def plot_MC_states(z_to_b,a_to_b,nu):
        plt.rcParams['axes.spines.right'] = True
        fig, ax = plt.subplots(figsize=(10,10))
        sns.set_palette("Paired",n_colors=12)
        z1 = np.logspace(-1,2,100)
        for a in [0.1,0.2,0.4,0.8,1,1.5,2,3,5,10,100]:
            ax.plot(fR(z1,a,1,nu=nu),-z1,label='$%.1f$' %a)
            ax.plot(fR(z1,a,1),-z1,ls='--')
        
        f = fR(z_to_b,a_to_b,1,nu=nu)
        ax.set_xlim(left=0)
        ax.set_ylim(-100,-0.1)
        ax.grid(which='both')
        ax.set_yscale('symlog',linthresh=0.01)
        ax.set_yticks([-0.1,-1,-10,-100])
        ax.set_yticklabels([0.1,1,10,100])
        ax.set_yticks(-np.logspace(-1,2,10), minor=True)
        ax.set_xlabel('$f_\\mathrm{R}$')
        ax.set_ylabel('$z$ / $b$')

        ax.plot(fR(z1,a_to_b,1,nu=nu),-z1,label='$a$ / $b = %.1f$' %a_to_b,color='black')
        ax.plot([0,f,f],[-z_to_b,-z_to_b,-100],color='black',ls=':')
        ax.plot(f,-z_to_b,marker='o',color='red')
        ax.text(0.8,-2,'$s = %.6f \\sigma_0 b/E$' %f)
        fig.legend(bbox_to_anchor=(1.15, 0.8))
        #fig.tight_layout()
        ax.plot(0,-0.1,color='black',ls='-',label='$\\nu = %.1f$' %nu)
        ax.plot(0,-0.1,color='black',ls='--',label='$\\nu = 0$')
        plt.show();

In [ ]:
def interactive_fK_rechteck_nu():
    @interact(z_to_b=widgets.BoundedFloatText(value=1, min=0.1, max=100., description='$z$ / $b$'),
              a_to_b=widgets.BoundedFloatText(value=2, min=0.1, max=100., description='$a$ / $b$')
    )
    def plot_MC_states(z_to_b,a_to_b):
        plt.rcParams['axes.spines.right'] = True
        fig, ax = plt.subplots(figsize=(10,10))
        sns.set_palette("Paired",n_colors=12)        
        ax2 = ax.twiny()
        z1 = np.logspace(-1,2,100)
        z2 = np.logspace(-1,0,100)
        for a in [0.1,0.2,0.4,0.8,1,1.5,2,3,5,10,100]:
            ax.plot(fK(z1,a,1),-z1,label='$%.1f$' %a)
            ax2.plot(fK(z2,a,1),-z2)
        
        f = fK(z_to_b,a_to_b,1)
        ax.set_xlim(left=0)
        ax.set_ylim(-100,-0.1)
        ax2.set_xlim(left=0)
        ax.grid(which='both')
        ax.set_yscale('symlog',linthresh=0.01)
        ax.set_yticks([-0.1,-1,-10,-100])
        ax.set_yticklabels([0.1,1,10,100])
        ax.set_yticks(-np.logspace(-1,2,10), minor=True)
        ax.set_xlabel('$f_\\mathrm{K}$')
        ax2.set_xlabel('$f_\\mathrm{K}$')
        ax.set_ylabel('$z$ / $b$')

        if (z_to_b < 1):
            ax2.plot(fK(z2,a_to_b,1),-z2,label='$a$ / $b = %.1f$' %a_to_b,color='black',ls='--')
            ax2.plot([0.,f,f],[-z_to_b,-z_to_b,-0.1],color='black',ls=':')
            ax2.plot(f,-z_to_b,marker='o',color='red')
        else: 
            ax.plot(fK(z1,a_to_b,1),-z1,label='$a$ / $b = %.1f$' %a_to_b,color='black',ls='--')
            ax.plot([0,f,f],[-z_to_b,-z_to_b,-100],color='black',ls=':')
            ax.plot(f,-z_to_b,marker='o',color='red')
        ax.text(2,-2,'$s_\\mathrm{K} = %.6f \\sigma_0 b/E^*$' %f)
        ax2.spines['bottom'].set_position(('data', -0.1))
        ax2.xaxis.set_label_position('top') 
        ax2.xaxis.tick_top()
        fig.legend(bbox_to_anchor=(1.15, 0.8))
        #fig.tight_layout()
        plt.show();

In [4]:
def run_layers(update_layers):
    global items
    items = []
    for i in range(n_layers.value):
        items.append(widgets.FloatText(description='$z_{%i}$ / m' %i,value=1+i,min=0))
        items.append(widgets.FloatText(description='$E_\\mathrm{s%i}$ / MPa' %i,value=(i+1)*10,min=100))
    return widgets.GridBox(items, layout=widgets.Layout(grid_template_columns="repeat(2,400px)"))   

In [ ]:
def run_analysis(update_analysis):
    global items
    global n_layers
    for i in range(n_layers.value):
        print(items[i].description, items[i].value)
        print(items[i+1].description, items[i+1].value)